This notebook show an example on how to run predictions using a pre-trained a lgsegmentor.

This example it uses the validation data 

In [1]:
import numpy as np
import tifffile
import napari
import leopardgecko.segmentor as lgs

import logging
logging.basicConfig(level=logging.INFO)

# Load data  for predcition and respective labels

In [2]:
data_to_pred = tifffile.imread("test_data/TS_0005_crop_val.tif")
labels_to_compare = tifffile.imread("test_data/TS_0005_ribos_membr_crop_val.tif")

In [3]:
print(data_to_pred.shape)

(256, 256, 256)


# Setup lg segmentor

In [4]:
#Create the class
lgsegmentor0 = lgs.cMultiAxisRotationsSegmentor.create_from_model("2023-09-29_ribos_membr.lgsegmod")

Check default settings and modify as needed

In [5]:
lgsegmentor0.NN1_pred_settings

namespace(quality='high',
          output_probs=True,
          clip_data=True,
          st_dev_factor=2.575,
          data_hdf5_path='/data',
          cuda_device=0,
          downsample=False,
          one_hot=False,
          prediction_axis='Z')

Set CUDA device.

To change cuda device don't change with .cuda_device, but rather use setcudadevice(devnumber)

You may want to check usage and memory-usage with command `!nvidia-smi` to see what is available before starting training

In [6]:
!nvidia-smi

Wed Oct 11 14:04:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:00:05.0 Off |                    0 |
| N/A   35C    P0    24W / 250W |      4MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:00:06.0 Off |                    0 |
| N/A   

In [7]:
lgsegmentor0.set_cuda_device(0)

Run the prediction and collect result

In [8]:
pred_labels = lgsegmentor0.predict(data_to_pred)

INFO:root:Setting up NN1 prediction
INFO:root:number of data sets to predict: 1
INFO:root:Data to predict index:0
INFO:root:Calculating mean of data...
INFO:root:Mean value: 3.376452088356018
INFO:root:Calculating standard deviation.
INFO:root:Std dev: 22.28497505498702. Calculating stats.
INFO:root:Lower bound: -54.00735867823556, upper bound: 60.7602628549476
INFO:root:Number of voxels above upper bound to be clipped 95283 - percentage 0.568%
INFO:root:Number of voxels below lower bound to be clipped 175185 - percentage 1.044%
INFO:root:Rescaling intensities.
INFO:root:Data is already in integer dtype, converting to float for rescaling.
INFO:root:Converting to uint8.
INFO:root:Loading model dictionary from file.
INFO:root:Sending the U-Net model to device 0
INFO:root:Loading in the saved weights.
INFO:root:Volume to be rotated by 0 degrees
INFO:root:Predicting YX slices:
INFO:root:Free GPU memory is 31.66 GB. Batch size will be 4.
INFO:root:Predicting segmentation for volume of shape

[########################################] | 100% Completed | 22.17 ss


INFO:root:NN2 prediction complete.


# Visualise result

In [9]:
import napari
NV=napari.Viewer()
NV.add_image(data_to_pred)
NV.add_labels(labels_to_compare)
NV.add_labels(pred_labels, color={1:'green'})

<Labels layer 'pred_labels' at 0x7efa502dfb80>

# Compare against ground truth

Use leopardgecko metrics to get voxel-F1 (dice score)

In [10]:
import leopardgecko.metrics as lgm

In [11]:
lgm.MetricScoreOfVols_Dice(pred_labels,labels_to_compare)

INFO:root:MetricScoreOfVols_Dice()
INFO:root:Number of class segmentations in first volume 3
INFO:root:Number of class segmentations in second volume 3
INFO:root:Calculating using numpy
INFO:root:dicescores = {1: 0.5188051691479704, 2: 0.5507384751326172}


(0.5347718221402937, {1: 0.5188051691479704, 2: 0.5507384751326172})

# Run the prediction in blocks and compare with previous result

In [12]:
import leopardgecko.utils as lgu

In [13]:
pred_labels_block = lgu.map_vol_function_by_blocking(lgsegmentor0.predict, data_to_pred, (120,120,120),(20,32,40))

INFO:root:BLOCK: New block, intended origin iz0,iy0,ix0 = 0,0,0 , use origin iz00,iy00,ix00 = 0,0,0 , end iz1,iy1,ix1 = 120,120,120
INFO:root:BLOCK: Start calculation with this block
INFO:root:Setting up NN1 prediction
INFO:root:number of data sets to predict: 1
INFO:root:Data to predict index:0
INFO:root:Calculating mean of data...
INFO:root:Mean value: 3.322815972222222
INFO:root:Calculating standard deviation.
INFO:root:Std dev: 21.626499424272946. Calculating stats.
INFO:root:Lower bound: -52.365420045280615, upper bound: 59.01105198972506
INFO:root:Number of voxels above upper bound to be clipped 8998 - percentage 0.521%
INFO:root:Number of voxels below lower bound to be clipped 18442 - percentage 1.067%
INFO:root:Rescaling intensities.
INFO:root:Data is already in integer dtype, converting to float for rescaling.
INFO:root:Converting to uint8.
INFO:root:Loading model dictionary from file.
INFO:root:Sending the U-Net model to device 0
INFO:root:Loading in the saved weights.
INFO:r

[########################################] | 100% Completed | 2.33 sms


INFO:root:NN2 prediction complete.
INFO:root:BLOCK: This block's calculation completed
INFO:root:BLOCK:Crop block result from origin jz0,jy0,jx0 = : 0,0,0
INFO:root:BLOCK:Copying cropped block to datares
INFO:root:BLOCK: First block result initialises datares
INFO:root:BLOCK: New block, intended origin iz0,iy0,ix0 = 0,0,80 , use origin iz00,iy00,ix00 = 0,0,80 , end iz1,iy1,ix1 = 120,120,200
INFO:root:BLOCK: Start calculation with this block
INFO:root:Setting up NN1 prediction
INFO:root:number of data sets to predict: 1
INFO:root:Data to predict index:0
INFO:root:Calculating mean of data...
INFO:root:Mean value: 3.4229733796296298
INFO:root:Calculating standard deviation.
INFO:root:Std dev: 20.795109456706907. Calculating stats.
INFO:root:Lower bound: -50.12443347139066, upper bound: 56.97038023064992
INFO:root:Number of voxels above upper bound to be clipped 11469 - percentage 0.664%
INFO:root:Number of voxels below lower bound to be clipped 15958 - percentage 0.923%
INFO:root:Rescalin

[########################################] | 100% Completed | 2.54 sms


INFO:root:NN2 prediction complete.
INFO:root:BLOCK: This block's calculation completed
INFO:root:BLOCK:Crop block result from origin jz0,jy0,jx0 = : 0,0,20
INFO:root:BLOCK:Copying cropped block to datares
INFO:root:BLOCK: New block, intended origin iz0,iy0,ix0 = 0,0,160 , use origin iz00,iy00,ix00 = 0,0,136 , end iz1,iy1,ix1 = 120,120,256
INFO:root:BLOCK: Start calculation with this block
INFO:root:Setting up NN1 prediction
INFO:root:number of data sets to predict: 1
INFO:root:Data to predict index:0
INFO:root:Calculating mean of data...
INFO:root:Mean value: 3.346806712962963
INFO:root:Calculating standard deviation.
INFO:root:Std dev: 20.454237638503237. Calculating stats.
INFO:root:Lower bound: -49.32285520618287, upper bound: 56.016468632108804
INFO:root:Number of voxels above upper bound to be clipped 10388 - percentage 0.601%
INFO:root:Number of voxels below lower bound to be clipped 17535 - percentage 1.015%
INFO:root:Rescaling intensities.
INFO:root:Data is already in integer d

...

INFO:root:NN2 prediction complete.
INFO:root:BLOCK: This block's calculation completed
INFO:root:BLOCK:Crop block result from origin jz0,jy0,jx0 = : 10,16,20
INFO:root:BLOCK:Copying cropped block to datares
INFO:root:BLOCK: New block, intended origin iz0,iy0,ix0 = 200,176,160 , use origin iz00,iy00,ix00 = 136,136,136 , end iz1,iy1,ix1 = 256,256,256
INFO:root:BLOCK: Start calculation with this block
INFO:root:Setting up NN1 prediction
INFO:root:number of data sets to predict: 1
INFO:root:Data to predict index:0
INFO:root:Calculating mean of data...
INFO:root:Mean value: 3.445274884259259
INFO:root:Calculating standard deviation.
INFO:root:Std dev: 21.15562393651363. Calculating stats.
INFO:root:Lower bound: -51.03045675226335, upper bound: 57.92100652078186
INFO:root:Number of voxels above upper bound to be clipped 9391 - percentage 0.543%
INFO:root:Number of voxels below lower bound to be clipped 16635 - percentage 0.963%
INFO:root:Rescaling intensities.
INFO:root:Data is already in in

[########################################] | 100% Completed | 2.44 sms


INFO:root:NN2 prediction complete.
INFO:root:BLOCK: This block's calculation completed
INFO:root:BLOCK:Crop block result from origin jz0,jy0,jx0 = : 10,16,20
INFO:root:BLOCK:Copying cropped block to datares
INFO:root:BLOCK: New block, intended origin iz0,iy0,ix0 = 200,176,240 , use origin iz00,iy00,ix00 = 136,136,136 , end iz1,iy1,ix1 = 256,256,256
INFO:root:BLOCK: Start calculation with this block
INFO:root:Setting up NN1 prediction
INFO:root:number of data sets to predict: 1
INFO:root:Data to predict index:0
INFO:root:Calculating mean of data...
INFO:root:Mean value: 3.445274884259259
INFO:root:Calculating standard deviation.
INFO:root:Std dev: 21.15562393651363. Calculating stats.
INFO:root:Lower bound: -51.03045675226335, upper bound: 57.92100652078186
INFO:root:Number of voxels above upper bound to be clipped 9391 - percentage 0.543%
INFO:root:Number of voxels below lower bound to be clipped 16635 - percentage 0.963%
INFO:root:Rescaling intensities.
INFO:root:Data is already in in

[########################################] | 100% Completed | 2.43 sms


INFO:root:NN2 prediction complete.
INFO:root:BLOCK: This block's calculation completed
INFO:root:BLOCK:Crop block result from origin jz0,jy0,jx0 = : 10,16,20
INFO:root:BLOCK:Copying cropped block to datares
INFO:root:BLOCK: Completed. Results should be in datares


In [15]:
import napari
NV=napari.Viewer()
NV.add_image(data_to_pred)
NV.add_labels(labels_to_compare)
NV.add_labels(pred_labels, color={1:'green'})
NV.add_labels(pred_labels_block, color={1:'red'})

<Labels layer 'pred_labels_block' at 0x7efb50f19c70>

Prediction with blocking algorithm is worse from the calculation without blocking so be careful when using it. Thre are other instances where it may work better

Compare "block"-prediction with ground truth

In [16]:
lgm.MetricScoreOfVols_Dice(pred_labels_block,labels_to_compare)

INFO:root:MetricScoreOfVols_Dice()
INFO:root:Number of class segmentations in first volume 3
INFO:root:Number of class segmentations in second volume 3
INFO:root:Calculating using numpy
INFO:root:dicescores = {1: 0.5113926075053858, 2: 0.4386024434511359}


(0.4749975254782609, {1: 0.5113926075053858, 2: 0.4386024434511359})

Compare "block"-prediction with whole prediction

In [17]:
lgm.MetricScoreOfVols_Dice(pred_labels_block,pred_labels)

INFO:root:MetricScoreOfVols_Dice()
INFO:root:Number of class segmentations in first volume 3
INFO:root:Number of class segmentations in second volume 3
INFO:root:Calculating using numpy
INFO:root:dicescores = {1: 0.9049133055788577, 2: 0.7928801885870097}


(0.8488967470829337, {1: 0.9049133055788577, 2: 0.7928801885870097})